<a href="https://colab.research.google.com/github/meizhong986/WhisperJAV/blob/main/notebook/WhisperJAV_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎌 WhisperJAV - Google Colab Edition
Generate Subtitles for Japanese Adult Videos using Free GPU

## 📋 Quick Start Guide
1. **Configure Your Settings** in the panel below (Cell 1).
2. Click `Runtime` → `Run all` in the menu to start everything.
3. **Connect Google Drive** when prompted.
4. The notebook will run all steps and disconnect automatically if the option is checked.

In [ ]:
#@title Configure & Run WhisperJAV
#@markdown ### ← Click here to hide the code and see only the options.
#@markdown Set your options below, then click `Runtime -> Run all`.

#@markdown --- 
#@markdown ### ⚙️ Core Settings
mode = "balanced"  #@param ["balanced", "fast", "faster"]
sensitivity = "balanced"  #@param ["balanced", "aggressive", "conservative"]
subs_language = "japanese"  #@param ["japanese", "english-direct"] 

#@markdown --- 
#@markdown ### ✨ Enhancement & Output Settings
adaptive_classification = False  #@param {type:"boolean"}
adaptive_audio_enhancement = False  #@param {type:"boolean"}
smart_postprocessing = True  #@param {type:"boolean"}
opening_prologue = "Subtitles by yourname" #@param {type:"string"}
closing_credits_text = "Subs by WhisperJAV Colab" #@param {type:"string"}

#@markdown --- 
#@markdown ### 🔌 Session Management
auto_disconnect = True #@param {type:"boolean"}
#@markdown *Automatically disconnect when finished to save GPU credits.*

#===============================================================================
#  ✅ END OF CONFIGURATION - THE REST OF THE NOTEBOOK IS AUTOMATED
#===============================================================================
import os
import sys
import subprocess
import shlex
import time
from pathlib import Path
import html
from google.colab import drive
from IPython.display import display, HTML
from tqdm.notebook import tqdm

print("--- STEP 1: PRE-FLIGHT CHECKS ---")
!nvidia-smi
print("✅ GPU check complete.\n")

print("--- STEP 2: CONNECTING GOOGLE DRIVE ---")
try:
    drive.mount('/content/drive', force_remount=True)
    drive_folder = Path('/content/drive/MyDrive/WhisperJAV')
    drive_folder.mkdir(exist_ok=True)
    print(f"✅ Google Drive connected. Using folder: {drive_folder}\n")
except Exception as e:
    display(HTML(f'<div style="background-color: #f8d7da; border: 1px solid #f5c6cb; border-radius: 8px; padding: 20px;"><h3 style="color: #721c24;">❌ ERROR: Failed to connect Google Drive.</h3><p style="color: #721c24;">Please re-run the cell and ensure you accept the authorization pop-up.</p></div>'))
    sys.exit()

print("--- STEP 3: INSTALLING DEPENDENCIES ---")

# Install system libraries
print("Installing system libraries...")
!apt-get update -qq > /dev/null
!apt-get install -y -qq ffmpeg > /dev/null

# Install WhisperJAV without dependencies (latest commit)
print("Installing WhisperJAV without dependencies...")
!pip install --no-deps -q git+https://github.com/meizhong986/WhisperJAV.git

# Define core dependencies with version constraints
core_requirements = [
    "openai-whisper@git+https://github.com/openai/whisper@v20250625",
    "stable-ts@git+https://github.com/meizhong986/stable-ts-fix-setup.git@main",
    "faster-whisper>=1.1.1",
    "ffmpeg-python",
    "soundfile",
    "auditok",
    "numpy<2.0",
    "scipy<2.0",
    "tqdm<5.0",
    "pysrt",
    "srt",
    "numba>=0.60.0"
]

# Check existing PyTorch installations
print("Checking PyTorch installations...")
try:
    import torch
    TORCH_AVAILABLE = True
    TORCH_VERSION = torch.__version__
    CUDA_AVAILABLE = torch.cuda.is_available()

    # Check for torchvision
    try:
        import torchvision
        TORCHVISION_AVAILABLE = True
        TORCHVISION_VERSION = torchvision.__version__
    except ImportError:
        TORCHVISION_AVAILABLE = False
    
    # Check for torchaudio
    try:
        import torchaudio
        TORCHAUDIO_AVAILABLE = True
        TORCHAUDIO_VERSION = torchaudio.__version__
    except ImportError:
        TORCHAUDIO_AVAILABLE = False

except ImportError:
    TORCH_AVAILABLE = False
    TORCHVISION_AVAILABLE = False
    TORCHAUDIO_AVAILABLE = False

# Install only if needed
install_torch = False
torch_message = "✅ Using existing: "
if not TORCH_AVAILABLE:
    install_torch = True
    torch_message += "Torch missing. "
elif not CUDA_AVAILABLE:
    install_torch = True
    torch_message += f"Torch CUDA unavailable ({TORCH_VERSION}). "
else:
    torch_message += f"Torch {TORCH_VERSION} with CUDA. "

if not TORCHVISION_AVAILABLE:
    install_torch = True
    torch_message += "TorchVision missing. "
elif TORCHVISION_AVAILABLE:
    torch_message += f"TorchVision {TORCHVISION_VERSION}. "

if not TORCHAUDIO_AVAILABLE:
    install_torch = True
    torch_message += "TorchAudio missing. "
elif TORCHAUDIO_AVAILABLE:
    torch_message += f"TorchAudio {TORCHAUDIO_VERSION}. "

if install_torch:
    print(torch_message)
    print("Installing PyTorch stack with CUDA support...")
    !pip install -q torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124
else:
    print(torch_message)

# Verify secondary dependencies
print("Checking secondary dependencies...")
dependencies = [
    "ffmpeg-python", "soundfile", "auditok", "numpy", "scipy",
    "tqdm", "pysrt", "srt", "numba", "faster-whisper"
]
missing_deps = []
for dep in dependencies:
    try:
        __import__(dep)
        # Special check for faster-whisper
        if dep == "faster-whisper":
            import faster_whisper
            if not hasattr(faster_whisper, "__version__") or faster_whisper.__version__ < "1.1.1":
                missing_deps.append(f"faster-whisper<1.1.1")
    except ImportError:
        missing_deps.append(dep)

if missing_deps:
    print(f"⚠️ Missing/outdated dependencies: {', '.join(missing_deps)}")
    print("Installing required dependencies...")
    # Install core dependencies including any missing ones
    !pip install -q {' '.join(core_requirements)}
else:
    print("✅ All secondary dependencies are satisfied")

print("✅ Dependencies installed.\n")

# If user doesn't change the example, treat it as empty
if opening_prologue == "Subtitles by yourname":
    opening_prologue = ""

print("--- STEP 4: RUNNING WHISPERJAV TRANSCRIPTION ---")

# Build the command robustly as a list of arguments
command_list = [
    'whisperjav',
    str(drive_folder)
]

options = {
    '--mode': mode,
    '--sensitivity': sensitivity,
    '--subs-language': subs_language,
    '--output-dir': str(drive_folder),
    '--adaptive-classification': adaptive_classification,
    '--adaptive-audio-enhancement': adaptive_audio_enhancement,
    '--smart-postprocessing': smart_postprocessing
}

for flag, value in options.items():
    if isinstance(value, bool):
        if value:
            command_list.append(flag)
    elif value:
        command_list.append(flag)
        command_list.append(str(value))

# Join the list into a shell-safe string to be used with Popen(shell=True)
full_command = shlex.join(command_list)
print(f"Executing command: {full_command}\n")

# Execute with live output and robust error handling
try:
    with subprocess.Popen(full_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, universal_newlines=True) as process:
        for line in process.stdout:
            print(line, end='')

    if process.returncode != 0:
        raise subprocess.CalledProcessError(process.returncode, process.args)

except subprocess.CalledProcessError as e:
    error_message = f"The main process failed with exit code {e.returncode}."
    display(HTML(f'''<div style="background-color: #f8d7da; border: 1px solid #f5c6cb; border-radius: 8px; padding: 20px;"><h3 style="color: #721c24;">❌ ERROR: Transcription Failed</h3><p style="color: #721c24;">{error_message} Please check the console output above for the specific error from the script.</p></div>'''))
    sys.exit()

print("\n--- STEP 5: POST-PROCESSING CREDITS ---")
srt_files = list(drive_folder.glob('*.srt'))

if opening_prologue:
    prologue_line = f"0\n00:00:00,000 --> 00:00:00,500\n{opening_prologue}\n\n"
    for srt_file in tqdm(srt_files, desc="Adding Opening Credits"):
        try:
            original_content = srt_file.read_text(encoding='utf-8')
            srt_file.write_text(prologue_line + original_content, encoding='utf-8')
        except Exception as e:
            print(f"  - Warning: Could not add prologue to {srt_file.name}: {e}")

if closing_credits_text:
    for srt_file in tqdm(srt_files, desc="Adding Closing Credits"):
        try:
            with open(srt_file, 'a', encoding='utf-8') as f:
                f.write(f'\n9999\n23:59:58,000 --> 23:59:59,000\n{closing_credits_text}\n')
        except Exception as e:
            print(f"  - Warning: Could not add closing credits to {srt_file.name}: {e}")

print("✅ Post-processing complete.\n")

display(HTML("""<div style=\"background-color: #d4edda; border: 1px solid #c3e6cb; border-radius: 8px; padding: 20px; margin-top: 20px;\"><h3 style=\"color: #155724; margin-top: 0;\">🎉 Success! All tasks are complete.</h3><p style=\"color: #155724; margin-bottom: 0;\">The session will now disconnect automatically if you enabled the option.</p></div>"""))

if auto_disconnect:
    print("\n🔌 Auto-disconnect enabled. This session will now end to save resources.")
    time.sleep(5)
    from google.colab import runtime
    runtime.unassign()